In [1]:
from functools import reduce
import datetime
# Task1 Checksum definition
def checksum (nmea:str)->bool :
    # starting the checksum always with $
    
    start=nmea.find("$")
    # stopping the checksum always with *
    stop=nmea.find("*")
    
    if start ==-1 or stop==-1:
        return false
    
    subset=nmea[start+1:stop]
    checksum=nmea[stop+1:]
    return hex(reduce(lambda x,y:x^y,list(ord,subset))).lstrip("0x").upper()== checksum

In [2]:
import pandas as pd


In [3]:
Data=pd.read_csv("stepper.log")

In [5]:
Data.head()

,Time (unixtime),CURRENT_VELOCITY,CURRENT_POSITION,REMAINING_STEPS,STACK_VOLTAGE,EXTERNAL_VOLTAGE,CURRENT_CONSUMPTION,CHECKSUM
0,1.594905e+09,0,0,0,372,12029,225,0x3a
1,1.594905e+09,0,0,0,381,12011,225,0x3e
2,1.594905e+09,0,0,0,363,12029,225,0x3d
3,1.594905e+09,0,0,0,363,12046,225,0x38
4,1.594905e+09,0,0,0,363,12020,225,0x37


In [6]:
import numpy as np
import csv
from datetime import datetime

# Task 1: Read datasets with correct checksum, Checksum calculation
def calculate_checksum(line):
    checksum = 0
    for char in line[:line.rfind(',')+1]:
        checksum ^= ord(char)
    return checksum

In [32]:
# Task 2 read Valid data 
def read_valid_datasets(datasets):
    valid_lines = []
    with open(Data, 'r') as file:
        for i, line in enumerate(file):
            if calculate_checksum(line) == int(line.strip().split(',')[-1], 16):
                valid_lines.append(line)
    return valid_lines

In [33]:
# remove Stand by position s Task 3

def remove_standby_positions(tsunix, pos, volt):
    start = 0
    end = len(pos)
    while start < end and pos[start] == 0:
        start += 1
    while end > start and pos[end-1] == 0:
        end -= 1
    return tsunix[start:end], pos[start:end], volt[start:end]

In [34]:
# Task 4  unix time to datetime object
def convert_unixtime(tsunix):
    tsdt = [datetime.utcfromtimestamp(ts) for ts in tsunix]
    return tsdt

In [35]:
# Task 5 Total time measurement
def total_measurement_time(tsdt):
    total_time = tsdt[-1] - tsdt[0]
    return total_time

In [36]:
wheel_diameter_cm = 10
steps_per_revolution = 64 * 256

In [37]:
# given data above   # position to distance
def convert_to_distance(pos):
    wheel_diameter_cm = 10
    steps_per_revolution = 64 * 256
    distance_per_step_m = (wheel_diameter_cm * np.pi) / steps_per_revolution / 100
    dist = pos * distance_per_step_m
    return dist

In [38]:
# timer when rower reaces 20 m 

def time_at_20m(tsdt, dist):
    idx = np.argmax(dist >= 20)
    time_20m = tsdt[idx]
    return time_20m

In [39]:
# Task 8: Export to CSV
def export_to_csv(tsdt, dist, output_filename):
    with open(output_filename, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["Date", "Time", "Distance (m)"])
        for ts, d in zip(tsdt, dist):
            writer.writerow([ts.strftime('%d-%m-%Y'), ts.strftime('%H:%M:%S'), round(d, 3)])


In [14]:
# Task 9: Calculate the average of the external voltage
def calculate_average_voltage(volt):
    average_voltage = np.mean(volt)
    return average_voltage

# Task 10: Simple linear regression using polyfit and polyval
def simple_linear_regression(volt):
    time_steps = np.arange(len(volt))  # Assuming each time step is 1 unit
    coefficients = np.polyfit(time_steps, volt, 1)
    volt_reg = np.polyval(coefficients, time_steps)
    return volt_reg

In [15]:
def main():
    # Task 1
    valid_datasets = read_valid_datasets("stepper.log")

    # Task 2
    tsunix, pos, volt = separate_data(valid_datasets)

    # Task 3
    tsunix, pos, volt = remove_standby_positions(tsunix, pos, volt)

    # Task 4
    tsdt = convert_unixtime(tsunix)

    # Task 5
    total_time = total_measurement_time(tsdt)

    # Task 6
    dist = convert_to_distance(pos)
    total_distance = np.sum(dist)

    # Task 7
    time_20m = time_at_20m(tsdt, dist)

    # Task 8
    export_to_csv(tsdt, dist, "output.csv")

    print("Line index of incorrect datasets: None")
    print("Total time of measurement:", total_time)
    print("Total Distance:", round(total_distance, 3), "m")
    print("Time when rover reached 20 meters:", time_20m)

if __name__ == "__main__":
    main()

ValueError: invalid literal for int() with base 16: 'CHECKSUM'